## LauzHack 2024 Mini hackathon on LLMs - Intro to LLM agent APIs
Luca Fusar Bassini

In [2]:
### old imports with other cool stuff, code will be made available

# for an application example of this collection of APIs, see our project PaperBoat, currently under dev here: https://github.com/JBil8/paperboat/tree/jacopo
# environment api-hackathon

# tutorial copypasted and edited from various online resources! not mine :) could be quite outdated now, as i made this notebook in October 2023

import requests
from flask import Flask, request, Response
#from scihub import SciHub
import os
import re
import random
import json
import datetime
from bs4 import BeautifulSoup
from pyzotero import zotero
from google.cloud import texttospeech as tts
import wave
import langchain
from crossref.restful import Works
from semanticscholar import SemanticScholar
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import scipy
import matplotlib.pyplot as plt
from typing import Sequence
import openai
import pandas as pd
import tiktoken
from openai.embeddings_utils import get_embedding
from dotenv import load_dotenv, find_dotenv
import warnings
warnings.filterwarnings("ignore")
import langchain
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI
from langchain.agents import tool
from datetime import date
from dotenv import load_dotenv, find_dotenv
from semanticscholar import SemanticScholar

In [3]:
# note: a few useful links
# https://learn.deeplearning.ai/
# https://platform.openai.com/docs/plugins/examples
# https://python.langchain.com/docs/additional_resources/tutorials
# https://github.com/openai/chatgpt-retrieval-plugin
# https://cookbook.openai.com/

In [4]:
# read in all the API keys

# OpenAI
try:
    with open('openai_key.txt', 'r') as file:
        openai_key = file.read()
except Exception as e:
        print(e)
        
try:
    with open('pinecone_key.txt', 'r') as file:
        pineconekey = file.read()
except Exception as e:
        print(e)

# Telegram
try:
    with open('telegram_key.txt', 'r') as file:
        TOKEN = file.read()
except Exception as e:
        print(e)

In [5]:
openai.api_key = openai_key

In [6]:
mykey = openai_key

## Agents: ChatGPT + LangChain

In [11]:
# langchain: agents that can search the web
llm_model = "gpt-3.5-turbo"

# create an instance of ChatGPT
llm = ChatOpenAI(temperature=0, model=llm_model, openai_api_key=mykey)

# parse useful tools, eg, Wikipedia, that the "agent" can use
tools = load_tools(["llm-math","wikipedia"], llm=llm)

agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [12]:
agent("What is the 25% of 300?") # look how cool is the behind the scenes to build new tools...



> Entering new AgentExecutor chain...
Thought: We can use the Calculator tool to find 25% of 300.
Action:
```
{
  "action": "Calculator",
  "action_input": "25% of 300"
}
```
Observation: Answer: 75.0
Thought:I now know the final answer

Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

In [13]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 



> Entering new AgentExecutor chain...
Thought: I should use Wikipedia to find out which book Tom M. Mitchell wrote.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Tom M. Mitchell"
}
```
Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as t

In [14]:
# make an agent that can write and EXECUTE inside it freshly-written, dynamic Python code!
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


We can use the `sorted()` function in Python to sort the list of customers based on last name first and then first name.
Action: Python_REPL
Action Input: customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
print(sorted_customers)
Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

Thought:The customers have been sorted by last name and then first name.
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

> Finished chain.


"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

In [15]:
# inspect the full behind the scenes - this is slow
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

In [16]:
# building langchain agents/tools with prompting 

@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")



> Entering new AgentExecutor chain...
Thought: I should use the `time` tool to find out today's date.

Action:
```
{
  "action": "time",
  "action_input": ""
}
```


Observation: 2024-04-16
Thought:Final Answer: 2024-04-16

> Finished chain.


## Structuring ChatGPT's "reasoning chain" & prompting

In [19]:
# "chain-of-thought" prompting: structuring LLM's activity to resemble human reasoning at structured tasks involving language
# note how you can use prompting creatively

def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products: 
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

# again note the steps the "chain"
user_message = f"""
by how much is the BlueWave Chromebook more expensive \
than the TechPro Desktop"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is comparing the prices of two specific products.
Step 2:#### The user is comparing the BlueWave Chromebook and the TechPro Desktop.
Step 3:#### The assumption made by the user is that the BlueWave Chromebook is more expensive than the TechPro Desktop.
Step 4:#### The TechPro Desktop is priced at $999.99, and the BlueWave Chromebook is priced at $249.99. Therefore, the BlueWave Chromebook is $750 cheaper than the TechPro Desktop.
Response to user:#### The BlueWave Chromebook is actually $750 cheaper than the TechPro Desktop.


## OpenAI vector embeddings

In [20]:
# ChatGPT vector embeddings

# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

# load & inspect dataset
input_datapath = "Reviews.csv" 
df = pd.read_csv(input_datapath, index_col=0)
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
df.head(2)

,Time,ProductId,UserId,Score,Summary,Text,combined
Id,,,,,,,
1,1303862400,B001E4KFG0,A3SGXH7AUHU8GW,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,Title: Good Quality Dog Food; Content: I have ...
2,1346976000,B00813GRG4,A1D87F6ZCVE5NK,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Title: Not as Advertised; Content: Product arr...


In [21]:
# subsample to 1k most recent reviews and remove samples that are too long
top_n = 1000
df = df.sort_values("Time").tail(top_n * 2)  # first cut to first 2k entries, assuming less than half will be filtered out
df.drop("Time", axis=1, inplace=True)

encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)

df = df.iloc[1:20,]
df

,ProductId,UserId,Score,Summary,Text,combined,n_tokens
Id,,,,,,,
220697,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...,35
107908,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,"Title: It isn't blanc mange, but isn't bad . ....",267
107800,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,Title: These also have SALT and it's not sea s...,239
205313,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,Title: Happy with the product; Content: My dog...,86
284450,B008PSM0BQ,A3OUFIMGL2K6RS,4,Good Sauce,This is a good all purpose sauce. Has good fl...,Title: Good Sauce; Content: This is a good all...,107
378704,B008YA1LQK,A9YEAAQVHFUTX,5,Blackcat,Great coffee! Love all Green Mountain coffee ...,Title: Blackcat; Content: Great coffee! Love ...,34
456909,B001KP6B98,ABWCUS3HBDZRS,5,Excellent product,After scouring every store in town for orange ...,Title: Excellent product; Content: After scour...,100
378703,B008YA1LQK,A2RSB6FVQ9K9OD,5,Bulk k-Cups,This is the best way to buy coffee for my offi...,Title: Bulk k-Cups; Content: This is the best ...,44
312410,B001E5E2QI,A23WYVBCNE75X1,3,It's Okay,"Next time, I will buy Gevalia Irish Cream deca...","Title: It's Okay; Content: Next time, I will b...",38


In [22]:
# this may take a few minutes
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))

In [23]:
df.to_csv("fine_food_reviews_with_embeddings_1k.csv")

df

,ProductId,UserId,Score,Summary,Text,combined,n_tokens,embedding
Id,,,,,,,,
220697,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...,35,"[-0.023657305166125298, -0.011915579438209534,..."
107908,B000JMBE7M,AQX1N6A51QOKG,4,"It isn't blanc mange, but isn't bad . . .",I'm not sure that custard is really custard wi...,"Title: It isn't blanc mange, but isn't bad . ....",267,"[0.0001991958706639707, 0.005254689138382673, ..."
107800,B004AHGBX4,A2UY46X0OSNVUQ,3,These also have SALT and it's not sea salt.,I like the fact that you can see what you're g...,Title: These also have SALT and it's not sea s...,239,"[0.010382632724940777, -0.013499382883310318, ..."
205313,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had ...,Title: Happy with the product; Content: My dog...,86,"[0.015346125699579716, -0.003973433747887611, ..."
284450,B008PSM0BQ,A3OUFIMGL2K6RS,4,Good Sauce,This is a good all purpose sauce. Has good fl...,Title: Good Sauce; Content: This is a good all...,107,"[-0.016191359609365463, 0.005975224543362856, ..."
378704,B008YA1LQK,A9YEAAQVHFUTX,5,Blackcat,Great coffee! Love all Green Mountain coffee ...,Title: Blackcat; Content: Great coffee! Love ...,34,"[-0.01409711129963398, -0.016294565051794052, ..."
456909,B001KP6B98,ABWCUS3HBDZRS,5,Excellent product,After scouring every store in town for orange ...,Title: Excellent product; Content: After scour...,100,"[0.0006881337030790746, -0.017053602263331413,..."
378703,B008YA1LQK,A2RSB6FVQ9K9OD,5,Bulk k-Cups,This is the best way to buy coffee for my offi...,Title: Bulk k-Cups; Content: This is the best ...,44,"[-0.002279900014400482, -0.0001825799117796123..."
312410,B001E5E2QI,A23WYVBCNE75X1,3,It's Okay,"Next time, I will buy Gevalia Irish Cream deca...","Title: It's Okay; Content: Next time, I will b...",38,"[-0.017475998029112816, -0.01266452670097351, ..."


In [24]:
# how long are the review embeddings?
# we could now treat each review as a 1536-features element of a vector space to do clusterings etc!!
# much more here: https://platform.openai.com/docs/guides/embeddings/use-cases

print(len(df["embedding"].iloc[1,]))
print(len(df["embedding"].iloc[17,]))

1536
1536


# Search over pdf documents with LangChain + others

In [34]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

#loader = UnstructuredPDFLoader("field-guide-to-data-science.pdf")
loader = OnlinePDFLoader("https://www.biorxiv.org/content/10.1101/2023.07.20.549945v1.full.pdf")

data = loader.load_and_split() 

print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

print (f'Now you have {len(texts)} documents')

OPENAI_API_KEY = mykey

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

import pinecone      

pinecone.init(      
	api_key=pineconekey,      
	environment='gcp-starter'      
)      
index = pinecone.Index('langchain')

index_name = "langchain"  ####### I0VE STARTED A VERY RANDOM INDEX ON FREE TIER IN PINECONE!!

docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

query = "what is Voyager?"
docs = docsearch.similarity_search(query)#, include_metadata=True)

print(docs)

ImportError: cannot import name 'open_filename' from 'pdfminer.utils' (/Users/fusar/anaconda3/envs/api_hackathon/lib/python3.8/site-packages/pdfminer/utils.py)

In [39]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

llm = OpenAI(temperature=0, openai_api_key=mykey) ### temperature parameter controls the "creativity" (aka, hallucinations)
chain = load_qa_chain(llm, chain_type="stuff")

query = "summarize the technical improvements in Voyager"
docs = docsearch.similarity_search(query)#, include_metadata=True)

chain.run(input_documents=docs, question=query)

AttributeError: module 'docsearch' has no attribute 'similarity_search'

In [37]:
pip install docsearch

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 29.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.8 MB/s eta 0:00:00
  Created wheel for docsearch: filename=docsearch-1.0-py3-none-any.whl size=4403 sha256=ef62e476af49bfeb2d82601f69668c334e82688afc7568cd6f5b76728826cd5b
  Stored in directory: /Users/fusar/Library/Caches/pip/wheels/7b/67/17/84cd2974bfcceb566031ea8aaafe7878bfa7df321b0b6931dd
  Created wheel for pyemd: filename=pyemd-1.0.0-cp38-cp38-macosx_11_0_arm64.whl size=182816 sha256=4b0216fbea23e8122d6b646229618c5800f9fa9e746f70351e5acab88d5a47ff
  Stored in directory: /Users/fusar/Library/Caches/pip/wheels/f1/70/0e/06a1f01d960b40b01077a2c9c6